In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor

In [47]:
train_df = pd.read_csv('./data/train.csv', index_col='id').drop(['city', 'osm_city_nearest_name', 'date'], axis=1)
train_df = train_df[pd.to_numeric(train_df['floor'], errors='coerce').notnull()]
train_df['floor'] = pd.to_numeric(train_df['floor'], errors='coerce')
train_df = train_df.fillna(0.0)
# train_df['street'] = [int(s[1:]) for s in train_df['street']]

In [48]:
train_df['street'] = [s if type(s) == float else float(s[1:]) for s in train_df['street']]
# print(train_df['street'])
# train_df['street'].head(100).to_numpy()
# train_df.astype(float)

In [68]:
# dict(train_df.dtypes)
# train_df.dtypes.to_numpy()
regions = train_df['region'].unique()
reg_dict = dict(zip(regions, range(len(regions))))
train_df['region'] = [reg_dict[r] for r in train_df['region']]
train_df = train_df.astype(float)

In [ ]:
cat_params = {}

model_cat = CatBoostClassifier(iterations=1000, 
                           task_type="GPU")